In [1]:
import pandas as pd
import os

print(os.getcwd())
print(os.path.exists("ifeval/results/SmolLM2-135M-SFT-5/eval_results_strict.jsonl"))
print(os.path.exists("ifeval/results/SmolLM2-135M/eval_results_strict.jsonl"))

best_dir = "ifeval/results/SmolLM2-135M-SFT-5/eval_results_strict.jsonl"
base_dir = "ifeval/results/SmolLM2-135M/eval_results_strict.jsonl"

best_df = pd.read_json(best_dir, lines=True)
base_df = pd.read_json(base_dir, lines=True)

print(os.path.exists("ifeval/results/SmolLM2-135M-SFT-5/eval_results_strict_base.jsonl"))
print(os.path.exists("ifeval/results/SmolLM2-135M/eval_results_strict_ft4.jsonl"))



def explode_and_compute_accuracy(df: pd.DataFrame) -> pd.DataFrame:
    df = df.explode(["instruction_id_list", "follow_instruction_list"]).reset_index(drop=True)
    acc = (
        df.groupby("instruction_id_list")["follow_instruction_list"]
          .apply(lambda x: sum(x) / len(x))
          .reset_index()
          .rename(columns={"follow_instruction_list": "accuracy"})
    )
    return acc

def compare_accuracy(base_df: pd.DataFrame, best_df: pd.DataFrame):
    # Compute for both dataframes
    base_acc = explode_and_compute_accuracy(best_df)
    best_acc = explode_and_compute_accuracy(base_df)

    # Merge and compute accuracy difference
    comparison = pd.merge(
        base_acc, best_acc, on="instruction_id_list", how="outer", suffixes=("_base", "_best")
    ).fillna(0)

    comparison["accuracy_diff"] = comparison["accuracy_best"] - comparison["accuracy_base"]
    comparison["improve"] = comparison["accuracy_diff"] > 0

    return comparison 

c:\Users\lamye\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Python 3.9\COMP4901B_LLM\COMP4901B-LLMs-main\assignment2
True
True
False
False


In [2]:

compare_accuracy(best_df, base_df).sort_values("accuracy_diff", ascending=False)

,instruction_id_list,accuracy_base,accuracy_best,accuracy_diff,improve
7,detectable_format:constrained_response,0.000000,1.000000,1.000000,True
6,detectable_content:postscript,0.166667,1.000000,0.833333,True
3,combination:repeat_prompt,0.000000,0.777778,0.777778,True
4,combination:two_responses,0.000000,0.666667,0.666667,True
12,keywords:existence,0.250000,0.750000,0.500000,True
11,detectable_format:title,0.166667,0.666667,0.500000,True
0,change_case:capital_word_frequency,0.142857,0.571429,0.428571,True
18,length_constraints:number_paragraphs,0.000000,0.400000,0.400000,True
19,length_constraints:number_sentences,0.200000,0.600000,0.400000,True
20,length_constraints:number_words,0.285714,0.571429,0.285714,True
